### Собственная реализация

In [1]:
import sys
sys.path.append('../')
sys.path.append('./')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from tqdm import tqdm

from collections import defaultdict
from collections import OrderedDict

import time
import timeit

import json

from utils.utils import plot_acc_and_time
from utils.config import DATASET_PATH
from models import SVM

%matplotlib inline
sns.set(style='darkgrid', font_scale=1.5)

In [4]:
data = pd.read_csv('../'+DATASET_PATH, index_col=[0, 1])
data = data.reset_index(drop=True)
data = data.drop(['position', 'age'], axis=1)
data.head()

,x_mean,x_std,x_max,x_min,x_energy,x_iqr,y_mean,y_std,y_max,y_min,...,x_iqr_pos1,y_iqr_pos1,z_iqr_pos1,xy_corr_pos1,xz_corr_pos1,yz_corr_pos1,height,mass,is_valid,sex
0,11156.76,123.532772,11380.0,10836.0,1.244884e+08,177.0,1158.80,73.172275,1348.0,920.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
1,11156.88,128.858961,11380.0,10836.0,1.244924e+08,193.0,1137.96,69.416023,1300.0,920.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
2,11154.36,134.410311,11532.0,10836.0,1.244376e+08,190.0,1088.32,171.829072,1380.0,164.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
3,10998.28,415.506022,11532.0,9384.0,1.211331e+08,323.0,853.36,499.190851,1380.0,-1032.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
4,10883.04,409.575930,11532.0,9384.0,1.186066e+08,543.0,537.68,622.192456,1380.0,-1032.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1


In [5]:
y = data['is_valid']
X = data.drop('is_valid', axis=1)

Так как для обучения модели и её хранения нужно малое кол-во памяти, то возьмем 30% от выборки. Это не должно сказаться на точности, так как имеется множество повторяющхися элементов выборки. Также будем использовать float16.\\

Точность без квантизации и урезания данных - 0.99

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.3, random_state=1234
)

max_iter = 10000
C = 30

In [7]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train).astype(np.half)
y_train = y_train.to_numpy(dtype=np.half)

In [8]:
svc = SVM(kernel='rbf', C=C, max_iter=10000)
svc.fit(X_train, y_train)

100%|██████████| 10000/10000 [1:13:16<00:00,  2.27it/s]


In [9]:
X_test = scaler.transform(X_test).astype(np.half)
y_test = y_test.to_numpy(dtype=np.half)

In [10]:
y_pred = svc.predict(X_test)

In [11]:
accuracy_score(y_pred, y_test)

0.953555657073245